Primero cargamos todas las librerias que usaremos,en conjunto con los PATHS de las bases

In [1]:
import numpy as np
import random
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.stats.stattools import durbin_watson
import math
from scipy.stats import norm
#Cambiar el path *Importante
PATH_2210 = "E01_T_DEU_CONS_2210_REV.csv"
PATH_2211 = "E01_T_DEU_CONS_2211_REV.csv"
PATH_2212 = "E01_T_DEU_CONS_2212_REV.csv"
PATH_2301 = "E01_T_DEU_CONS_2301_REV.csv"
PATH_2302 = "E01_T_DEU_CONS_2302_REV.csv"
PATH_2303 = "E01_T_DEU_CONS_2303_REV.csv"
Lista_Path = [PATH_2210,PATH_2211,PATH_2212,PATH_2301,PATH_2302,PATH_2303]

In [2]:
def RutComunes(lista):
    dataframelist = []
    for i in lista:
        df = pd.read_csv(i, delimiter=";", usecols = ['RUT'])
        dataframelist.append(df)

    interseccion = np.array(dataframelist[0]['RUT'])

    for df in dataframelist[1:]:
        RutComunes = np.intersect1d(interseccion, df['RUT'])
        
    dfRutComunes = pd.DataFrame({'RUT': RutComunes})
    lista_rut = dfRutComunes["RUT"]
    return lista_rut

0                0
1                1
2                3
3                4
4                6
            ...   
1028140    1083121
1028141    1083122
1028142    1083123
1028143    1083124
1028144    1083125
Name: RUT, Length: 1028145, dtype: int64

Generamos el Test de Durbin Watson y el de tamaño muestra

In [50]:
def DataProm(dataframe):
    cantidad_filas = dataframe.shape[0]
    tamaño_muestra =calcular_tamano_muestra(cantidad_filas,0.05,0.5,0.05)
    lista_dataframes =[]
    for i in range (1000):
        dataframe_muestra = dataframe.sample(tamaño_muestra)
        lista_dataframes.append(dataframe_muestra)

    #print(lista_dataframes[30])
    num_columnas = lista_dataframes[0].shape[1]
    a = 1
    nuevas_fechas = {i: str(a + i) for i in range(num_columnas-1)}
    lista_fechas = list(nuevas_fechas.values())
    lista_durbinwatson = []
    for j in lista_dataframes:
        lista_durbinwatsonfecha = []
        for fecha in lista_fechas:
            if int(fecha) != len(lista_fechas):
                dw_test = TestSeriesDW(j,int(fecha),int(fecha)+1)
            lista_durbinwatsonfecha.append(dw_test)
        lista_durbinwatson.append(lista_durbinwatsonfecha)
    lista_final = []

    for lista in lista_durbinwatson:
        nueva_lista = lista[:-1]  # Eliminar el último elemento de la lista
        lista_final.append(nueva_lista)
    return lista_final



def TestSeriesDW (dataframe,fecha_1,fecha_2):

    df_fecha_1 = dataframe[str(fecha_1)]
    df_fecha_2 = dataframe[str(fecha_2)]
    dw_test = durbin_watson(df_fecha_2 - df_fecha_1)
    return dw_test


def calcular_tamano_muestra(N, alpha, p, d):
    z = abs(norm.ppf(1 - alpha/2)) 
    numerator = (N * z**2 * p * (1 - p))
    denominator = (d**2 * (N - 1) + z**2 * p * (1 - p))
    n = numerator / denominator
    muestra_tamaño = math.ceil(n)
    return muestra_tamaño


In [56]:
def Test(Variable,lista):
    lista_rut=RutComunes(Lista_Path)
    ###########################################################################
    dataframelist = []
    dataframelistna = []
    for i in lista:
        df1 = pd.read_csv(i, delimiter=";", usecols = ['RUT',Variable])
        df2 = df1.loc[df1['RUT'].isin(lista_rut)].dropna()
        dataframelist.append(df1)
        dataframelistna.append(df2)
    
    
    dataframes=[]
    a=1
    for i in dataframelistna:
        tupla = (i,str(a))
        dataframes.append(tupla)
        a += 1
    dftemporal = pd.DataFrame()
   
    
    for df, nombre_df in dataframes:
        dftemporal[nombre_df] = df[Variable]

    dftemporal['RUT'] = dataframelist[0]['RUT'] 
    dftemporal = dftemporal.dropna()

    
    dataframeDW = DataProm(dftemporal)
    Matrix = np.array(dataframeDW)
    valor_minimo = 1.72789
    valor_maximo = 1.80942
    mayores = np.mean(Matrix >= valor_maximo) * 100
    menores = np.mean(Matrix <= valor_minimo) * 100
    media = np.mean((Matrix >= valor_minimo) & (Matrix <= valor_maximo)) * 100

    print("Porcentaje de elementos mayores o iguales a", valor_maximo, ":", mayores)
    print("Porcentaje de elementos menores o iguales a", valor_minimo, ":", menores)
    print("Porcentaje de elementos entre", valor_minimo, "y", valor_maximo, ":", media)

    NewMatrix =  np.abs(Matrix - 4)

    may = np.mean(NewMatrix>= valor_maximo) * 100
    men = np.mean(NewMatrix <= valor_minimo) * 100
    med = np.mean((NewMatrix >= valor_minimo) & (NewMatrix <= valor_maximo)) * 100
    print("Porcentaje de elementos mayores o iguales a", valor_maximo, ":", may)
    print("Porcentaje de elementos menores o iguales a", valor_minimo, ":", men)
    print("Porcentaje de elementos entre", valor_minimo, "y", valor_maximo, ":", med)

Finalmente se llega a calcular Dw segun las fechas

In [57]:
Test("DIASMORA",Lista_Path)

Porcentaje de elementos mayores o iguales a 1.80942 : 96.96000000000001
Porcentaje de elementos menores o iguales a 1.72789 : 0.27999999999999997
Porcentaje de elementos entre 1.72789 y 1.80942 : 2.76
Porcentaje de elementos mayores o iguales a 1.80942 : 97.58
Porcentaje de elementos menores o iguales a 1.72789 : 0.2
Porcentaje de elementos entre 1.72789 y 1.80942 : 2.22
